# Flask REST API in Jupyter runs as a Server

In [1]:
from flask import Flask

UPLOAD_FOLDER = 'static/uploads/'


app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
import numpy as np 
import pandas as pd 
from PIL import Image
import tensorflow as tf
import random
from matplotlib import pyplot as plt
import os
import h5py

In [2]:
savedModel=tf.keras.models.load_model('./model.h5')
savedModel.summary()

2023-02-01 13:08:50.238516: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-01 13:08:50.238885: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 170, 272, 30)      2280      
                                                                 
 batch_normalization_6 (Batc  (None, 170, 272, 30)     120       
 hNormalization)                                                 
                                                                 
 re_lu_6 (ReLU)              (None, 170, 272, 30)      0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 169, 271, 30)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 85, 136, 30)       8130      
                                                                 
 batch_normalization_7 (

In [3]:
def predict(imag):
    img = (Image.open(imag).resize((819,512)) )   
    potMes = ""
    val = np.asarray(img)

    plt.imshow(val)

    swapped = np.moveaxis(val, 0, 2)  # shape (y_pixels, x_pixels, n_bands)
    arr4d = np.expand_dims(val,0)
    prediction = int(savedModel.predict(arr4d))
    if prediction < 15:
        potMes = "We suggest you should avoid burning this area"
    elif prediction > 85:
        potMes = "There is evident residue in this area, consider redirecting it to other byproducts"   
    else: potMes = ""
    return(f"We predict a residue of {int(savedModel.predict(arr4d))}%" + potMes)

In [ ]:
import os

import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

ALLOWED_EXTENSIONS = set(['jpg','jpeg'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def upload_form():
        return render_template('home1.html')


@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
       
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('Please enter image')
        return render_template('home2.html')
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        
        message = predict(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        print(message)
        return render_template('home2.html', filename=filename, output = message)
    else:
        flash('Only jpg or jpeg images are allowed')
        return render_template('home2.html')

@app.route('/display/<filename>')
def display_image(filename):
   
    return redirect(url_for('static', filename='uploads/' + filename), code=301)
@app.route('/home2')
def home2():
    return render_template('home2.html')
@app.route('/home3')
def home3():
    return render_template('home3.html')

@app.route('/home1')
def home1():
        return render_template('home1.html')


if __name__ == "__main__":
    app.run(port = 5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Feb/2023 13:08:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:09:16] "GET /home3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:09:20] "GET /home2 HTTP/1.1" 200 -
2023-02-01 13:09:25.020126: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-01 13:09:25.131225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [01/Feb/2023 13:09:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:09:25] "GET /static/uploads/ABoVE_alder_photo_2019.jpg HTTP/1.1" 200 -


We predict a residue of 24%
1/1 [==============================] - 0s 12ms/step


127.0.0.1 - - [01/Feb/2023 13:09:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:09:46] "GET /static/uploads/ABoVE_alder_photo_2019.jpg HTTP/1.1" 200 -


We predict a residue of 24%


127.0.0.1 - - [01/Feb/2023 13:09:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:10:17] "GET /home1 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:10:20] "GET /home3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 13:10:22] "GET /home2 HTTP/1.1" 200 -


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5600
Press CTRL+C to quit
127.0.0.1 - - [31/Jan/2023 19:05:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2023 19:05:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/Jan/2023 19:05:55] "POST /f HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2023 19:05:55] "GET /display/static/uploads/FileStorage_918_and_LaFerrari.jpg_image_jpeg HTTP/1.1" 404 -


static/uploads/FileStorage_918_and_LaFerrari.jpg_image_jpeg
../static/usrimgs/FileStorage_918_and_LaFerrari.jpg_image_jpeg
